# Popularity

## Experiment

- Using Jaccard Similarity has the best performance

In [1]:
import glob
import numpy as np
import pandas as pd
from typing import Callable, Tuple
from collections import Counter
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

from tensorflow.keras.utils import get_file
import zipfile

In [2]:
def load_data(data_size : str) -> pd.DataFrame:
    ''' load Movie Lens data '''

    if data_size == '100k':
        file = get_file('ml-100k.zip', 'http://files.grouplens.org/datasets/movielens/ml-100k.zip')
        file_name = 'ml-100k/*'
    elif data_size == '1m':
        file = get_file('ml-1m.zip', 'http://files.grouplens.org/datasets/movielens/ml-1m.zip')
        file_name = 'ml-1m/ratings.dat'
    elif data_size == '10m':
        file = get_file('ml-10m.zip', 'http://files.grouplens.org/datasets/movielens/ml-10m.zip')
        file_name = 'ml-10M100K/ratings.dat'
    elif data_size == '20m':
        file = get_file('ml-20m.zip', 'http://files.grouplens.org/datasets/movielens/ml-20m.zip')
        file_name = 'ml-20m/ratings.csv'
    elif data_size == '25m':
        file = get_file('ml-25m.zip', 'http://files.grouplens.org/datasets/movielens/ml-25m.zip')
        file_name = 'ml-25m/ratings.csv'
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()

    col_names = ['userId', 'movieId', 'rating', 'timestamp']
    if data_size in ['20m', '25m']:
        ratings = pd.read_csv(file_name, engine = 'python')
    else:
        ratings = pd.read_csv(file_name, sep = '|', delimiter = '::', names = col_names, engine = 'python')
    print(ratings.shape)
    return ratings

In [3]:
ratings = load_data('10m')
ratings.head()

65568768/65566137 [==============================] - 1s 0us/step
(10000054, 4)


,userId,movieId,rating,timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


## 1. Data Preprocessing

In [4]:
def preprocessing(df: pd.DataFrame, threshold = 4) -> pd.DataFrame:
    df = df[df.rating >= threshold]
    positive = df.groupby('userId')['movieId'].nunique()
    positive = positive.index[positive >= 5]
    df = df[df.userId.isin(positive)]
    return df.reset_index(drop = True)

In [5]:
idx_user_map = ratings.userId.unique()
user_idx_map = {e: i for i, e in enumerate(idx_user_map)}
n_user = idx_user_map.shape[0]
print(f'# of user = {n_user}')

idx_item_map = ratings.movieId.unique()
item_idx_map = {e: i for i, e in enumerate(idx_item_map)}
n_item = idx_item_map.shape[0]
print(f'# of item = {n_item}')

# of user = 69878
# of item = 10677


In [6]:
def Id2idx(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(userId = lambda x: x.userId.map(user_idx_map), 
                     movieId = lambda x: x.movieId.map(item_idx_map))

def idx2Id(df: pd.DataFrame) -> pd.DataFrame:
    return df.assign(userId = lambda x: x.userId.apply(lambda x: idx_user_map[x]), 
                     movieId = lambda x: x.movieId.apply(lambda x: idx_item_map[x]))

In [7]:
ratings = preprocessing(ratings)
ratings = Id2idx(ratings)

print(ratings.shape)
ratings.head(5)

(5003786, 4)


,userId,movieId,rating,timestamp
0,0,0,5.0,838985046
1,0,1,5.0,838983525
2,0,2,5.0,838983392
3,0,3,5.0,838983421
4,0,4,5.0,838983392


In [8]:
def train_valid_test_split(df: pd.DataFrame) -> pd.DataFrame:
    train_user, test_user = train_test_split(df.userId.unique(), test_size = 0.1, random_state = 7777)
    train, test = map(lambda x: df[df.userId.isin(x)], (train_user, test_user))
    train, test = map(lambda df: df.reset_index(drop = True), (train, test))
    return train, test

def query_answer_split(df: pd.DataFrame) -> pd.DataFrame:
    query, answer = train_test_split(df, test_size = 0.2, stratify = df.userId, random_state = 7777)
    query, answer = map(lambda df: df.sort_values(['userId', 'timestamp']), (query, answer))
    query, answer = map(lambda df: df.reset_index(drop = True), (query, answer))
    return query, answer

def list_aggregation(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('userId', as_index = False)[['movieId', 'rating']].agg(list)

In [9]:
class ItemCF:
    def __init__(self, N: int):
        self.N = N       
    
    def fit(self, df: pd.DataFrame):
        self.df = df
        row = df.userId
        col = df.movieId
        data = np.ones(shape = df.index.size)
        self.mat = csr_matrix((data, (row, col)), shape = (n_user, n_item))
        self.mat_T = self.mat.T.tocsr()
    
    def predict(self, df:pd.DataFrame, similarity = 'dot') -> list:
        res = []
        top_N = 20
        profile = df['movieId']
        for idx in tqdm(df.index):
            y_true = np.zeros(shape = (n_item, ))
            pos = profile[idx]
            y_true[pos] = 1

            sim = self.mat.dot(y_true)
            if similarity == 'cosine':
                a = self.mat.dot(np.ones(n_item))
                b = np.sum(y_true)
                sim /= np.clip(np.sqrt(a * b), a_min = 1, a_max = np.inf)
            elif similarity == 'jaccard':
                a = self.mat.dot(np.ones(n_item))
                b = np.sum(y_true)
                sim /= (a + b - sim)
            elif similarity == 'euclidean':
                a = self.mat.dot(np.ones(n_item))
                b = np.sum(y_true)
                sim = np.sqrt(a + b - 2 * sim)
            y_pred = self.mat_T.dot(sim)
            cand = np.argsort(np.where(y_true == 1, -10, 1) * y_pred)[:-top_N-1:-1]
            res.append(cand)
        return res

In [18]:
def ndcg(gt, rec):
    dcg, idcg = 0.0, 0.0
    k = 0
    for i, r in enumerate(rec):
        if r in gt:
            dcg += 1.0 / np.log(i + 2)
            idcg += 1.0 / np.log(k + 2)
            k += 1
    return dcg / np.max([1, idcg])

def recall(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(gt)

def precision(gt, rec):
    res = [r for r in rec if r in gt]
    return len(res) / len(rec)

def precision_k(gt, rec, k):
    rec_k = rec[:k+1]
    res = [r for r in rec_k if r in gt]
    return len(res) / k

def AP_k(gt, rec, k = 20):
    res = 0
    for i in range(k):
        if rec[i] in gt:
            res += precision_k(gt, rec, i+1)
    return res / min(k, len(gt))

In [19]:
def evaluation(true: pd.DataFrame, pred):
    m_ndcg = 0
    m_recall = 0
    m_precision = 0
    map_k = 0
    for i in range(len(pred)):
        rec = pred[i]
        gt = true.at[i, 'movieId']
        m_ndcg += ndcg(gt, rec)
        m_recall += recall(gt, rec)
        m_precision += precision(gt, rec)
        map_k += AP_k(gt, rec)
    m_ndcg /= len(pred)
    m_recall /= len(pred)
    m_precision /= len(pred)
    map_k /= len(pred)
    return m_ndcg, m_recall, m_precision, map_k

In [20]:
train, test = train_valid_test_split(ratings)
test_q, test_a = query_answer_split(test)
test_q, test_a = map(list_aggregation, (test_q, test_a))

In [21]:
model = ItemCF(N = 20)
model.fit(train)

### 0. Remark

- Since A and B are binary vectors, it satisfies $ A ^2 = A $ by the element-wise product. 

- Futher, note that the dot product "$\cdot$" satisfies that $ A \cdot B = \sum AB $.

### 1. Dot Product Similarity

$$ \text{DotSim}(A, B) := A \cdot B$$

In [16]:
pred = model.predict(test_q)

In [22]:
m_ndcg, m_recall, m_precision, map_k = evaluation(test_a, pred)
print(f'nDCG: {m_ndcg:.5f}',
      f'\nRecall: {m_recall:.5f}',
      f'\nPrecision: {m_precision:.5f}',
      f'\nMAP: {map_k:.5f}')

nDCG: 0.41426 
Recall: 0.19793 
Precision: 0.10392 
MAP: 0.10155


### 2. Cosine Similarity

$$ \text{CosSim}(A, B) := \cos(\angle AB) = \frac{A \cdot B}{|A||B|} = \frac{A \cdot B}{\sqrt{\sum A}\sqrt{\sum B}}$$

In [23]:
pred = model.predict(test_q, similarity = 'cosine')

In [25]:
m_ndcg, m_recall, m_precision, map_k = evaluation(test_a, pred)
print(f'nDCG: {m_ndcg:.5f}',
      f'\nRecall: {m_recall:.5f}',
      f'\nPrecision: {m_precision:.5f}',
      f'\nMAP: {map_k:.5f}')

nDCG: 0.42104 
Recall: 0.21662 
Precision: 0.10852 
MAP: 0.10816


### 3. Jaccard Similarity

$$ \text{JacSim}(A, B) := \frac{A \cdot B}{\sum A + \sum B + A \cdot B}$$

In [26]:
pred = model.predict(test_q, similarity = 'jaccard')

In [27]:
m_ndcg, m_recall, m_precision, map_k = evaluation(test_a, pred)
print(f'nDCG: {m_ndcg:.5f}',
      f'\nRecall: {m_recall:.5f}',
      f'\nPrecision: {m_precision:.5f}',
      f'\nMAP: {map_k:.5f}')

nDCG: 0.42903 
Recall: 0.22766 
Precision: 0.11146 
MAP: 0.11431


### 4. Euclidean Distance

$$
\begin{aligned}
\text{Dist}(A, B) &:= \sqrt{\sum(A - B)^2}
\\ &= \sqrt{\sum (A^2 - 2AB + B^2)}
\\ &= \sqrt{\sum (A + B - 2AB)}
\\ &= \sqrt{\sum A + \sum B - 2 A \cdot B}
\end{aligned}
$$

In [28]:
pred = model.predict(test_q, similarity = 'euclidean')

In [29]:
m_ndcg, m_recall, m_precision, map_k = evaluation(test_a, pred)
print(f'nDCG: {m_ndcg:.5f}',
      f'\nRecall: {m_recall:.5f}',
      f'\nPrecision: {m_precision:.5f}',
      f'\nMAP: {map_k:.5f}')

nDCG: 0.34237 
Recall: 0.15273 
Precision: 0.08366 
MAP: 0.06639
